In [1]:
import requests
from bs4 import BeautifulSoup
import csv

base_url = "https://www.snapdeal.com/products/sports-bicycles?sort=plrty&q=Seniority_s%3AAdult%7C#bcrumbLabelId:46102452"
num_pages = 10
all_product_data = []
total_items_scraped = 0

for page in range(1, num_pages + 1):
    url = base_url + str(page)

    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        product_listings = soup.find_all('div', class_='product-tuple-description')
        page_product_data = []
        for product in product_listings:
            product_name_tag = product.find('p', class_='product-title')
            product_name = product_name_tag.get('title') if product_name_tag else 'N/A'

            brand_name = product_name.split(' ', 1)[0]

            original_price_tag = product.find('span', class_='lfloat product-desc-price strike')
            discounted_price_tag = product.find('span', class_='lfloat product-price')

            original_price = original_price_tag.text.strip() if original_price_tag else 'N/A'
            discounted_price = discounted_price_tag.text.strip() if discounted_price_tag else 'N/A'

            original_price = original_price.split(' ', 1)[1].replace(',', '').strip() if original_price != 'N/A' else 'N/A'
            discounted_price = discounted_price.split(' ', 1)[1].replace(',', '').strip() if discounted_price != 'N/A' else 'N/A'

            original_price = float(original_price) if original_price != 'N/A' else 0
            discounted_price = float(discounted_price) if discounted_price != 'N/A' else 0

            percentage_discount = ((original_price - discounted_price) / original_price) * 100

            page_product_data.append([brand_name, product_name, original_price, discounted_price, percentage_discount])

        items_on_page = len(page_product_data)
        total_items_scraped += items_on_page

        all_product_data.extend(page_product_data)

    else:
        print(f"Failed to retrieve page {page}. Status code:", response.status_code)

with open('snapdeal_cycle.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Brand Name', 'Product Name', 'Original Price', 'Discounted Price', 'Percentage Discount'])
    csv_writer.writerows(all_product_data)

print("Data from all pages has been scraped and saved to snapdeal_cycle.csv")

Data from all pages has been scraped and saved to snapdeal_cycle.csv
